In [ ]:
%pylab inline

In [ ]:
import os
import pandas as pd

In [ ]:
num_samples = 20
num_segments = 200
min_num_snps = 3
outlier_rate = 0.01
noise_floor = 0.1
no_deletions = 1

mean_coverage = 50
alt_minor_fraction = 0.5

In [ ]:
def plot_vs_index(quantities, ylimits=None, yaxis='', thick_axhlines=[], thin_axhlines=[], **kwargs):
    figsize(20, 4)
    scatter(range(len(quantities)), quantities, **kwargs)
    for line in thick_axhlines:
        axhline(line, lw=2)
    for line in thin_axhlines:
        axhline(line, lw=0.25)
    xlim([0, len(quantities)])
    if ylimits is not None:
        ylim(ylimits)
    xlabel('index', fontsize=14)
    ylabel(yaxis, fontsize=14)
    
def alt_prob(minor_allele_fraction, is_alt_minor, ref_alt_bias_ratio, is_outlier):
    #uniform outlier + 2 x binomial mixture
    return is_outlier * random.uniform(0, 1, len(minor_allele_fraction)) + \
           (1 - is_outlier) * \
           ((minor_allele_fraction == 0) * \
            (is_alt_minor * random.uniform(0, noise_floor, len(minor_allele_fraction)) + \
             (1 - is_alt_minor) * (1 - random.uniform(0, noise_floor, len(minor_allele_fraction)))) + \
           (minor_allele_fraction != 0) * \
            (is_alt_minor / (1 + ref_alt_bias_ratio * (1. / (minor_allele_fraction + 1E-6) - 1)) + \
             (1 - is_alt_minor) / (1 + ref_alt_bias_ratio / (1. / (minor_allele_fraction + 1E-6) - 1))))

In [ ]:
#create 'samples/' directory to store output
if not os.path.exists('samples/'):
    os.mkdir('samples')

alpha_beta_output = []
for sample in range(num_samples):
    #number of snps per segment is chosen from GAMMA distribution
    num_snps_per_segment = min_num_snps + np.round(random.gamma(0.5, 250, num_segments)).astype(int64)
    num_snps = sum(num_snps_per_segment)
    
    #parameters for gamma (allele-fraction bias); gamma ~ 2 / (2 + GAMMA(alpha, beta))
    #see below for examples of distributions of mean(gamma) over samples and of gamma over snps within samples
    alpha = random.uniform(0.01, 50)
    beta = 0.05 / alpha * random.uniform(0.01, 5)
    ref_alt_bias_ratio_per_snp = 1 + random.gamma(alpha, beta, num_snps)
    mean_bias = mean(2. / (1 + ref_alt_bias_ratio_per_snp))
    
    #ref and alt copy numbers are drawn from GAMMA distributions peaked at 1
    ref_number_per_segment = np.round(0.25 + random.gamma(1.5, 1.25, num_segments))
    alt_number_per_segment = np.round(0.25 + random.gamma(1.5, 1.25, num_segments))
    if no_deletions:
        #noise model for deletions is not decided, so we can ignore them for now
        ref_number_per_segment = maximum(1., ref_number_per_segment)
        alt_number_per_segment = maximum(1., alt_number_per_segment)
    copy_number_per_segment = ref_number_per_segment + alt_number_per_segment
    
    true_minor_allele_fraction_per_segment = minimum(ref_number_per_segment, alt_number_per_segment) / \
                                                 maximum(1., copy_number_per_segment)
        
    true_minor_allele_fraction_per_snp = repeat(true_minor_allele_fraction_per_segment, num_snps_per_segment)
    is_outlier_per_snp = random.binomial(1, outlier_rate, num_snps)
    
    segment_index_per_snp = repeat(range(num_segments), num_snps_per_segment)

    reads_per_snp = random.poisson(mean_coverage, num_snps)
    is_alt_minor_per_snp = random.binomial(1, alt_minor_fraction, num_snps)
    alt_reads_per_snp = random.binomial(reads_per_snp, 
                                        alt_prob(true_minor_allele_fraction_per_snp, 
                                                 is_alt_minor_per_snp,
                                                 ref_alt_bias_ratio_per_snp,
                                                 is_outlier_per_snp))
    ref_reads_per_snp = reads_per_snp - alt_reads_per_snp
    alt_allele_fraction_per_snp = alt_reads_per_snp / reads_per_snp.astype(float64)
    minor_allele_fraction_per_snp = minimum(ref_reads_per_snp, alt_reads_per_snp) / reads_per_snp.astype(float64)
    
    allele_counts_pd = pd.DataFrame(zip(segment_index_per_snp, ref_reads_per_snp, alt_reads_per_snp),
                                    columns=['SEGMENT', 'REF_COUNT', 'ALT_COUNT'])
    allele_counts_pd.to_csv('samples/{:02d}'.format(sample) + '.tsv', sep='\t')
    
    alpha_beta_output += [[sample, alpha, beta, mean_bias]]
    
    plot_vs_index(alt_allele_fraction_per_snp, marker=".", alpha=0.5)
    title(r'sample {:02d}, $\alpha$ = {:2.2f}, $\beta$ = {:2.2e}, $\gamma$ = {:2.2f}'
          .format(sample, alpha, beta, mean_bias), fontsize=18)
    xlabel('SNP index')
    ylabel('alternate allele fraction')
    savefig('samples/{:02d}-AAF'.format(sample) + '.png')
    close()
#     plot_vs_index(minor_allele_fraction_per_snp, marker=".", alpha=0.5)

alpha_beta_pd = pd.DataFrame(alpha_beta_output,
                             columns=['SAMPLE', 'ALPHA', 'BETA', 'MEAN_BIAS'])
alpha_beta_pd.to_csv('samples/truth.tsv', sep='\t')

In [ ]:
num_dist_samples = 20
alpha_per_sample = random.uniform(0.01, 50, num_dist_samples)
beta_per_sample = 0.05 / alpha_per_sample * random.uniform(0.01, 5, num_dist_samples)
gamma_bias_per_snp_per_sample = [1 + random.gamma(alpha_per_sample[sample], beta_per_sample[sample], 200*100) 
                                 for sample in range(num_dist_samples)]
mean_gamma_bias_per_sample = [mean(2. / (1 + gamma_bias_per_snp)) 
                              for gamma_bias_per_snp in gamma_bias_per_snp_per_sample]

figsize(6, 4)
#show statistics and distribution for bias across samples
print 'all samples:'
print 'mean =', mean(mean_gamma_bias_per_sample), 'std =', std(mean_gamma_bias_per_sample)
print 'min =', min(mean_gamma_bias_per_sample), 'max =', max(mean_gamma_bias_per_sample)
hist(mean_gamma_bias_per_sample)
xlabel(r'$\langle\gamma\rangle_{snps}$', fontsize=16)
show()
print

#show statistics and distribution for bias across snps in a few individual samples
for sample, gamma_bias_per_snp in enumerate(gamma_bias_per_snp_per_sample[:5]):
    print 'sample ' + str(sample) + ':'
    print 'alpha =', alpha_per_sample[sample], 'beta =', beta_per_sample[sample]
    print 'mean =', mean(2. / (1 + gamma_bias_per_snp)), 'std =', std(2. / (1 + gamma_bias_per_snp))
    print 'min =', min(2. / (1 + gamma_bias_per_snp)), 'max =', max(2. / (1 + gamma_bias_per_snp))
    hist(2. / (1 + gamma_bias_per_snp), bins=20)
    xlabel(r'$\gamma$', fontsize=16)
    show()
    print
show()